In [1]:
from openpyxl import load_workbook
from google.cloud import storage
import pandas as pd
import pandasql as ps
import hashlib
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  

In [74]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("SampleSpark") \
    .getOrCreate()

In [75]:
mount_d ="/data/"

In [76]:
trgt_path = '/mnt/data_file.csv'
source_path = mount_d+'SPENDING HISTORY.xlsm'
sheet_name ='To_Backend'

In [77]:
def user_value(df):
    max_date = '1999-01-01'
    x= input("Enter 1 if it is adhoc else 0")
    str(x)
    if x == '1':
        return max_date
    
    elif x == '0':
        return df['Date'].max()
    else:
        print("Error: Invalid value for adhoc_load. Please use '0' or '1'.")
        user_value(df)
        

In [ ]:
df_max = pd.read_csv(trgt_path)
max_date = user_value(df_max)
print(max_date)


In [ ]:
df = pd.read_excel(source_path,sheet_name=sheet_name)
df = df[(pd.to_datetime(df['Date'])) >= pd.to_datetime(max_date)]
print(df)

In [ ]:

df['Month'] = df['Date'].dt.month_name()
df['Year'] = df['Date'].dt.year
df['Date'] = df['Date'].dt.date
df['Index'] = df.index
print(df)

In [81]:
# Function to create a hash from multiple columns
def hash_columns(*args):
    return hashlib.md5("".join(args).encode()).hexdigest()

# Create a primary key column
df['PKSK'] = df.apply(lambda row: hash_columns(str(row['Date']), str(row['Category']), str(row['Index'])), axis=1)

In [ ]:
print(df)

In [ ]:
duplicate_counts = df['PKSK'].value_counts()
duplicate_counts = duplicate_counts[duplicate_counts > 1]

# Fail the code if there are duplicates
if not duplicate_counts.empty:
    raise ValueError(f"Duplicate values found in 'col1':\n{duplicate_counts}")

# Proceed with the rest of the code if no duplicates
print("No duplicates found. Continuing execution...")

In [ ]:
trgt_df = pd.read_csv(trgt_path, header=0)
new_rows = df[~df['PKSK'].isin(trgt_df['PKSK'])]
df = pd.concat([df, new_rows],ignore_index=True)
print(f"Count of new rows: {len(new_rows)}")

In [ ]:
df.to_csv(trgt_path, mode= 'a',index=False,header=True)


In [ ]:
trgt_df = pd.read_csv(trgt_path, header=0)
print(trgt_df)